# The reference system

> **Goal**: Explain the reference system used for crystal orientation in FTIR and EBSD

## 1. The spherical coordinate system

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/4/4f/3D_Spherical.svg/1024px-3D_Spherical.svg.png" alt="image.png" style="zoom:55%;" />

Spherical coordinates $(r, \theta, \varphi)$ as used in physics (ISO 80000-2:2019), being $r$ the radial distance (distance to origin), polar angle $\theta$ (theta) (angle with respect to polar/zenith z-axis), and azimuthal angle $\varphi$ or $\phi$ (phi) (angle of rotation from the initial meridian plane).

> **Important note**: the angles $\theta$ and $\varphi$/$\phi$ in Asimov et al. (2006) are interchanged with respect to the convention used in physics (see https://en.wikipedia.org/wiki/Spherical_coordinate_system). To avoid confusion, perhaps it would be a good idea to rename them (alpha and beta?) since strictly speaking they are angles with respect to crystallographic axes, not to the reference system (although later we will make them coincide).

### 2. The FTIR reference frame

To establish a consistent coordinate frame and estimate the orientation of the crystals in FTIR we will have to consider the following:

- the microscope stage/sample pair
- the position of the reference transmittance envelope with respect to the reference system of the microscope
- the position of the crystal elements (axes, planes, etc.) with respect to the transmitance envelope

We start by defining that all coordinate systems used will be right handed, which implies the right hand rule for the sense of rotations (positive rotation counter-clockwise). To determine the full orientation of a crystal, it is necessary to define a Cartesian reference frame for the crystal lattice. The unit cell is commonly described using three lengths, $a$, $b$ and $c$ and three angles $α$, $β$, $γ$ as depicted in the figure below. For a general case:

- $a$, $b$ and $c$ form a right handed set.
- $c$ is always parallel to the $z$ axis.
- $b$ always lies in the $yz$ plane, at an angle $α$ respect to $c$
- $a$ is pointed such that it is an angle $β$ to $c$ and $γ$ to $b$

![EBSD_ref_frame](https://raw.githubusercontent.com/bertopadron/FTIR-EBSD/main/notebooks2learn/imgs/EBSD_unit_cell_convention.png)

As olivine crystal is orthorhombic, the axes defining the olivine unit cell are perpendicular to each other and thus the $a$-, $b$- and $c$-axes are parallel to $x$, $y$, and $z$, respectively. With this reference frame, the transmittance figure in our coordinate system is as follows:

TODO → ADD FIGURE

In [2]:
import sys
import matplotlib as mpl
from datetime import date    
today = date.today().isoformat()

print(f'Notebook tested in {today} using:')
print('Python', sys.version)
print('Matplolib', mpl.__version__)

Notebook tested in 2024-01-12 using:
Python 3.10.13 | packaged by Anaconda, Inc. | (main, Sep 11 2023, 13:15:57) [MSC v.1916 64 bit (AMD64)]
Matplolib 3.8.0
